## Vetle + Seb code

In [ ]:
%matplotlib inline

import os
import pandas as pd
import lasio as ls
#import selenium
#import phantomjs
import matplotlib.pyplot as plt
import numpy as np
import sys
import time
import dask.dataframe as dd
import dask.array as da
#import pyarrow.parquet as pq
import fastparquet
from functools import reduce

In [ ]:
import hvplot.pandas
import hvplot.dask
from hvplot import hvPlot
import holoviews as hv
from holoviews import opts, streams
from holoviews.plotting.links import DataLink
hv.extension('bokeh', logo=None)
from IPython.display import display, HTML
from bokeh.models.tools import LassoSelectTool, BoxSelectTool  # tools=['hover']
box_select = BoxSelectTool()
lasso_select = LassoSelectTool()
import panel as pn
pn.extension()
from IPython.display import display, HTML

client_secret = "Bw2w*31.CmbIiP.i2EILQ=HN@xr]yeu?"

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 80%; }
    div#maintoolbar-container { width: 80%; }
</style>
"""))

### Las generator

In [ ]:
filepath = '../Documentation/Well log LAS files/WKF W8A LWD VISION RM 0_5m Drilling.LAS'

In [ ]:
def las_to_df(filepath):
    las = ls.read(filepath)
    curve_df = las.df()
    curve_df = curve_df.replace(-999.25,np.nan)
    curve_df.reset_index(inplace=True)
    curve_df.rename(columns={curve_df.columns[0]:'Depth'}, inplace=True)
    las.curves[0].mnemonic = 'Depth'
    return curve_df, las

In [ ]:
df_realwell, las_realwell = las_to_df(filepath)

In [ ]:
# Create new df only containing relevant curves (9) within a start and stop depth:
def get_curves(df, curvenames, start_depth, stop_depth):
    df_new = df.loc[start_depth<=df['Depth']].loc[df['Depth']<stop_depth][curvenames]
    return df_new

In [ ]:
all_curves = ['Depth', 'GR_ARC', 'A16H', 'A28H', 'A40H', 'TNPH', 'ROP5_RM', 'RHOB', 'DCAV'] # Relevant curve names for the chosen well

start_depth = df_realwell['Depth'].iloc[0]
end_depth = df_realwell['Depth'].iloc[-1]
step_depth = df_realwell['Depth'].iloc[1] - df_realwell['Depth'].iloc[0]

df_curves = get_curves(df_realwell, all_curves, start_depth, end_depth) # New df containing specific curves and start and stop depths

#### Set input parameters

In [ ]:
Number_of_files = int(input("Number of files to generate: "))
depth_start = int(input("Depth reading start: "))
depth_end = int(input("Depth reading end: "))
depth_step = float(input("number of recordings per meter : ")) # 1=1 per m, 0.5=2 per m, 12= every 12 m
wellname = input('name of wells: ')
filename = input('name of files: ')

In [ ]:
len_data = len(np.arange(depth_start, depth_end, depth_step))

#### Create las-file from scratch

In [ ]:
def file_gen_input(wellname):
    file_arr = np.arange(1,Number_of_files+1,1)
    lasfile = ls.LASFile()
    from datetime import datetime
    lasfile.well.DATE = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    lasfile.well.WELL = wellname
    lasfile.params['CR8TOR'] = ls.HeaderItem('CR8TOR', value='Sebastian Aegidius')
    lasfile.other = 'LAS file creator'
    depth = np.arange(depth_start, depth_end, depth_step)
    lasfile.add_curve('DEPTH (m)', depth, unit = 'm')
    return lasfile, depth

In [ ]:
lasfile, depth = file_gen_input(wellname)

In [ ]:
def add_data(names, info_eq, fgi): # (Name of curve info, equation to produce data, file_gen_input_curve)
    data_eq = info_eq
    #fake[:3] = np.nan # Set first three values to NaN
    fgi.add_curve(names[0], data_eq, unit=names[1], descr=names[2]) # (parameter-name, datavalues, unit, description)
    return

#### Add multiple files, equations and data

In [ ]:
def multiple_files(Number_of_files, wellname = wellname):
    list_of_all_wells = []
    for i in range(Number_of_files):
        sample = file_gen_input(wellname+'_'+str(i))
        list_of_all_wells.append(sample[0])
    return list_of_all_wells

In [ ]:
params = [['GR', 'api', 'Gamma Ray'], 
          ['RD', 'ohm.m', 'Deep Resistivity'], 
          ['RM', 'ohm.m', 'Medium Resistivity'], 
          ['RS', 'ohm.m', 'Shallow Resistivity'], 
          ['NPHI', 'v/v', 'Neutron Porosity'], 
          ['RHOB', 'g/cm3', 'Density'], 
          ['DT', 'ms/ft', 'Sonic Velocity'], 
          ['CALI', 'inch', 'Caliper'], 
          ['BS', 'inch', 'Bit Size']
         ]

def eq_realdata(df):
    equation = [df[df.columns[0]].values, 
                df[df.columns[1]].values, 
                df[df.columns[2]].values,
                df[df.columns[3]].values, 
                df[df.columns[4]].values,
                df[df.columns[5]].values,
                df[df.columns[6]].values, 
                df[df.columns[7]].values, 
                df[df.columns[8]].values]
    equation = np.transpose(equation)
    equation = equation[0:len_data, :] # Only include equations from our chosen depth-interval
    return np.transpose(equation)

In [ ]:
def eq(lasfiles_object, eq_list):
    equations = []
    for i in range(len(lasfiles_object)): # Make unique equations for each las-file
        equations.append(eq_list)
    return equations

In [ ]:
def add_fakedata_multiple_files(fgi, equations, param_list):
    for las in range(len(fgi)): # Go through each las-file
        random = np.abs(np.random.gamma(shape=3, scale=2, size=len(depth))) # Random-data MUST be inside for-loop!
        #random_shift = 10 * np.random.sample() - 5
        random_shift = np.random.choice(np.concatenate([(1 + np.random.randn(5)*2), abs(np.random.randn(10)), (8 + np.random.randn(2) * 20)]))
        [add_data([param_list[i][0], param_list[i][1], param_list[i][2]], 
                      equations[las][i]*np.abs(np.random.gamma(shape=3, scale=2, size=len(depth)))
                      + equations[las][i]*random_shift, fgi[las]) for i in range(len(param_list))] # add 'random' data to a las-file
    return

In [ ]:
def write_to_las(fgi, filename):
    [fgi[i].write(filename+'_'+str(i)+'.las', version=2) for i in range(len(fgi))];
    return

In [ ]:
generated_wells = multiple_files(Number_of_files) # X number of las-files
add_fakedata_multiple_files(generated_wells, eq(generated_wells, eq_realdata(df_curves)), params) # Include some data in each file
write_to_las(generated_wells, filename) # Update las-files with the new data. TAKES ABOUT 20-30 SEC FOR 100 LAS-FILES.

#### Seb part:

In [ ]:
#File location and naming

def find_files(basedir, extfilter=''):
    files_ = []
    dirs = [basedir]
    while dirs:
        for e in os.scandir(dirs.pop()):
            if e.is_dir():
                dirs.append(e)
            else:
                if e.name.upper().endswith(extfilter.upper()):
                    files_.append(e.path)
    return files_

def path_leaf(path):
    for i in range(len(path)):
        path_name, file_name = os.path.split(path)
        file_name = file_name.replace('.las','')
    return file_name

def count_lines(filename):
    with open(filename) as f:
        return sum(1 for line in f)
    
if __name__ == '__main__':
    
    basedir = r"..\Documentation\Well log LAS files"
    start = time.perf_counter()
    files = find_files(basedir, '.las')
    stop = time.perf_counter()
    print('time to scan directory:', stop - start)
    print("Number of las files   :", len(files))
    print("Total lines           :", sum(map(count_lines, files)))
    #print(*files, sep='\n')
    #print()
    #file_name = [path_leaf(files[i]) for i in range(len(files))]
    #number_of_rows = [count_lines(files[i]) for i in range(len(files))]
    #print(*file_name, sep="\n")
    #print(*number_of_rows, sep='     ')
    #print()
    #print(number_of_rows[64])
    #print(file_name[64])

In [ ]:
def load_las_to_df(filepath_or_lasfile, generated=False):
    if (not generated):
        las = ls.read(filepath_or_lasfile)
        df = las.df()
    else:
        las = filepath_or_lasfile
        df = las.df() 
    df.reset_index(inplace=True)
    df.replace('-1.#IND', np.nan, inplace=True)
    df.replace('-1.#IO', np.nan, inplace=True)
    [df[x].astype('float64') for x in df.columns] # Convert all parameters to float64
    df.replace(-999.25, np.nan, inplace=True)
    df.rename(columns={df.columns[0]:'DEPTH'}, inplace=True)
    df['WELLNAME'] = las.well.WELL.value
    return df

In [ ]:
##### only to show the list of dataframes
list_of_wells_df = [load_las_to_df(files[i]) for i in range(len(files))]

In [ ]:
df = pd.concat(list_of_wells_df, ignore_index=True)
df

In [ ]:
#writing dataframe to csv file
df.to_csv("Test_df.csv")

#Writing dataframe to parquet file
df.to_parquet('Test_df.parquet', compression = 'gzip', engine='fastparquet)

### Create dask from las-generated files

In [ ]:
generated_wells_df = [load_las_to_df(generated_wells[i], generated=True) for i in range(len(generated_wells))]

In [ ]:
big_df = pd.concat(generated_wells_df, ignore_index=True)

In [ ]:
big_df['Category'] = np.concatenate([['cat'+str(i)]*20000 for i in range(int(len(big_df['WELLNAME'])/20000))]) # len(all_files)/20000 = 10 categories

In [ ]:
def get_numerics(df, depthcurvename): # only return the data that is numeric
    curve_list=list(df.columns[(df.dtypes.values == np.dtype('float64'))])
    curve_list.remove(depthcurvename) # Get rid of depth, so we have 9 remaining logs to plot
    return curve_list;

In [ ]:
num_curves = get_numerics(big_df, 'DEPTH')

In [ ]:
dask_dummy = dd.from_pandas(big_df, npartitions=len(big_df['Category'].unique()))

In [ ]:
dask_dummy.to_parquet('big_df.parquet', compression = 'gzip', engine='fastparquet')
dask_df = dd.read_parquet('big_df.parquet', engine='fastparquet') 

In [ ]:
dask_df = dask_df.set_index('WELLNAME').persist()

## Clustering

In [ ]:
import seaborn as sns
sns.set_color_codes()
import sklearn
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import scale # to scale our model

In [ ]:
list_files = [files[0], files[4], files[10], files[5]] # Choose three wells from our las-file folder
RHOB_names = ['RHOB', 'RHOZ', 'DEN', 'RHOB'] # Store the different density-names (NB: same order as list_files!)
NPHI_names = ['NPHI', 'NPHI', 'NPRL', 'NPHI'] # Store the different neutron-names (NB: same order as list_files!)
GR_names = ['GR', 'GR', 'GRGC', 'GR']

In [ ]:
def fill_NaN(df):
    for x in df.columns:
        if df[x].dtype == 'float64':
            len_NaN = len(df[x].isna())
            df[x].interpolate(method='linear', axis=0, limit=len_NaN, inplace=True) # Fill NaN values by doing linear interpolation with the preceding and upcoming value
            df[x].fillna(method='ffill', inplace=True)
            df[x].fillna(method='bfill', inplace=True)
            # Replace if any absurdly high or low values:
            if abs(np.min(df[x])) > 1000*np.mean(df[x]):
                df[x].replace(df[x].min(), np.mean(df[x]))
            if abs(np.max(df[x])) > 1000*np.mean(df[x]):
                df[x].replace(df[x].max(), np.mean(df[x]))
            df[x].fillna(np.mean(df[x]))
    return df

In [ ]:
# param1_names and param2_names must be a list of the two parameter names used by the files, in correct order!
def init_cluster(list_files, min_depth, max_depth, param1_names, param2_names, param1_realname, param2_realname):
    if (str(list_files[0]).lower().endswith('.las')): # if input is realwell
        df_and_las = [load_las_to_df(list_files[i]) for i in range(len(list_files))]
    else: # if input is generated well
        df_and_las = [load_las_to_df(list_files[i], generated=True) for i in range(len(list_files))]
    dfs = [df_and_las[i] for i in range(len(list_files))]
    
    for df in dfs:
        if ('DEPTH' not in df.columns):
            df.rename(columns={df.columns[0]:'DEPTH'}, inplace=True)
    [fill_NaN(i) for i in dfs]
    
    hold = []
    for df, param1_name, param2_name in zip(dfs, param1_names, param2_names):
        df = df.loc[df['DEPTH'] > min_depth]
        df = df.loc[df['DEPTH'] < max_depth]
        df.rename(columns={param1_name:param1_realname}, inplace=True)
        df.rename(columns={param2_name:param2_realname}, inplace=True)
        hold.append(df)
    dfs = hold
    return dfs, param1_realname, param2_realname

def KMeans_cluster(well_cluster):        
    multiple_wells = pd.concat(list(well_cluster[0]))
    
    new_df = pd.DataFrame({well_cluster[1]:multiple_wells[well_cluster[1]], well_cluster[2]:multiple_wells[well_cluster[2]]})
    new_array = np.array(new_df)
    
    X = scale(new_array)
    y = pd.concat([pd.Series(np.zeros(int(len(new_df)/3))), pd.Series(np.ones(int(len(new_df)/3))),
               pd.Series(np.ones(int(len(new_df)/3))*2)], ignore_index=True)
    
    clustering = KMeans(n_clusters=len(list_files))
    clustering.fit(X)
    clustering.labels_
    
    palette = sns.color_palette('deep', n_colors=np.unique(clustering.labels_).max() +1) # +1 because it returns index, but we need amount
    colors = [palette[x] for x in clustering.labels_]
    plt.scatter(x=new_df[well_cluster[1]], y=new_df[well_cluster[2]], color=colors)
    plt.xlabel(well_cluster[1])
    plt.ylabel(well_cluster[2])
    plt.title('Clustering of %d wells' %len(list_files))
    plt.show()
    return

In [ ]:
well_cluster = init_cluster(list_files, 1000, 2000, GR_names, NPHI_names, 'GR', 'NPHI')
plot_well_cluster = KMeans_cluster(well_cluster)

### Clustering with Dask

In [ ]:
import dask_ml.datasets
import dask_ml.cluster

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

def interactive_scatter_cluster(big_df, n_clusters=4):
    start = time.perf_counter()
    @interact(param1 = widgets.Dropdown(
                        options = num_curves,
                        value = num_curves[0], # Set first well as default value
                        description = 'Param 1:',
                        disabled = False),
              param2 = widgets.Dropdown(
                        options = num_curves,
                        value = num_curves[1],
                        description = 'Param 2:',
                        disabled = False),
              from_well_nr = widgets.BoundedIntText(
                                value=0,
                                min=0,
                                max=len(big_df['WELLNAME'].unique()),
                                step=5,
                                description='From well nr:',
                                disabled=False
                            ),
              to_well_nr = widgets.BoundedIntText(
                                value=len(big_df['WELLNAME'].unique()),
                                min=0,
                                max=len(big_df['WELLNAME'].unique()),
                                step=5,
                                description='To well nr:',
                                disabled=False
                            )
    )
    def dask_KMeans(param1, param2, from_well_nr, to_well_nr):
        from_well_idx = from_well_nr * len(big_df.loc[big_df['WELLNAME'] == random.choice(big_df['WELLNAME'].unique())])
        to_well_idx = to_well_nr * len(big_df.loc[big_df['WELLNAME'] == random.choice(big_df['WELLNAME'].unique())])
        
        ml_df = pd.DataFrame({param1:big_df.iloc[from_well_idx:to_well_idx][param1], param2:big_df.iloc[from_well_idx:to_well_idx][param2]}) # create a (machine-learning) df containing two parameters
        ml_dd = dd.from_pandas(ml_df, npartitions=dask_df.npartitions)
        ml_array = ml_dd.to_dask_array(lengths=True) # lengths=True computes chunk size so the array is ready for machine-learning
        
        km = dask_ml.cluster.KMeans(n_clusters=n_clusters)
        km.fit(ml_array)
        
        ml_dd = ml_dd.assign(x = km.labels_).persist()
        scatt = ml_dd.hvplot.scatter(param1, param2, datashade=True, c=km.labels_, cmap='Viridis', 
                                     title='Clustering of %d wells' %(to_well_nr - from_well_nr))
        out = pn.Row(scatt)
        return hv.output(out)
    stop = time.perf_counter()
    print('Time:', stop - start)
    return

In [ ]:
interactive_scatter_cluster(big_df)

### Section: Wellnames as columns

In [ ]:
# Wellnames as columns using PANDAS:
new_big_df = pd.DataFrame()

start = time.perf_counter()
for i in range(len(generated_wells)):
    param_col = np.concatenate([big_df.loc[big_df['WELLNAME'] == big_df['WELLNAME'].unique()[i]][param[0]].values for param in params])
    new_big_df[big_df['WELLNAME'].unique()[i]] = param_col
stop = time.perf_counter()
print('Timex:', stop - start)

In [ ]:
# Include depth and parameter columns
depth_col = np.concatenate([big_df.loc[big_df['WELLNAME'] == big_df['WELLNAME'].unique()[0]]['DEPTH (m)'].values for _ in params]) # Same depth for all wells
new_big_df['DEPTH (m)'] = depth_col
param_col = np.concatenate([[param[0]]*int(len(new_big_df)/len(params)) for param in params])
new_big_df['Param'] = param_col

In [ ]:
# Get neutron porosity values for 10th column (10th well):
new_big_df.loc[new_big_df['Param'] == 'NPHI'][new_big_df.columns[10]]

In [ ]:
new_big_df.to_parquet('new_big_df.parquet', compression = 'gzip', engine='fastparquet')
new_dask_df = dd.read_parquet('new_big_df.parquet', engine='fastparquet') 

### Compare speed of Dask and Pandas

In [ ]:
def get_numerics(df, depthcurvename): # only return the data that is numeric
    curve_list=list(df.columns[(df.dtypes.values == np.dtype('float64'))])
    curve_list.remove(depthcurvename) # Get rid of depth, so we have 9 remaining logs to plot
    return curve_list;

num_curves = [get_numerics(generated_wells_df[i], 'DEPTH (m)') for i in range(len(generated_wells_df))]

In [ ]:
parameters = new_dask_df['Param'].unique().compute()

In [ ]:
def curve_plot_numerics(log, df, depthname, group=None, alpha=1, colors='blue', height=600, width=300):
    aplot = df.hvplot(x=depthname, y=log, by='WELLNAME', groupby=group ,invert=True, flip_yaxis=True, shared_axes=True, alpha=alpha, color=colors,
                       height=height, width=width).opts(fontsize={'labels': 16,'xticks': 14, 'yticks': 14})
    return aplot;

In [ ]:
def curve_plot_dask(log, dask_df, depthname, wellname, group=None, alpha=1, colors='blue', height=600, width=300):
    plot_df = dd.merge(new_dask_df.loc[new_dask_df['Param'] == log][depthname].compute(), new_dask_df.loc[new_dask_df['Param'] == log][wellname].compute()).T
    aplot = plot_df.hvplot(x=depthname, y=wellname, 
                            groupby=group ,invert=True, flip_yaxis=True, 
                            shared_axes=False, alpha=alpha, color=colors, height=height, 
                            width=width).opts(fontsize={'labels': 16,'xticks': 14, 'yticks': 14})
    #bplot = hvPlot(pd.DataFrame([new_dask_df.loc[new_dask_df['Param'] == log][depthname].compute(), new_dask_df.loc[new_dask_df['Param'] == log][wellname].compute()]).T,
    #              invert=True, flip_yaxis=True, shared_axes=True, alpha=alpha, color=colors, height=height, 
    #              width=width, fontsize={'labels': 16,'xticks': 14, 'yticks': 14})
    return aplot;

## NOT WORKING:

In [ ]:
# Plot 1/4 of the wells:
#plotlists = [curve_plot_numerics(x, list_of_wells_df[i], 'DEPTH (m)') for i in range(int(len(list_of_wells_df)/20)) for x in num_curves[i]]
start = time.perf_counter()
plotlists = [curve_plot_numerics(x, generated_wells_df[0], 'DEPTH (m)') for x in num_curves[0]]
stop = time.perf_counter()
print('Time Pandas:', stop-start)

start = time.perf_counter()
plotlists_dask = [curve_plot_dask(log, new_dask_df, 'DEPTH (m)', 'DaskSpeed2_10') for log in parameters]
stop = time.perf_counter()
print('Time Dask:', stop-start)